# Problema da Mochila

In [131]:
import random
import copy

## Definição do Problema
Você fará uma viagem a um acampamento durante o final de semana e precisa decidir quais itens levar. Como você só dispõe de uma mochila com capacidade para 15 kg, decidiu incluir somente os itens que maximizem a soma do valor em R$ dos itens, sem ultrapassar o limite de peso.  
Resolva o problema utilizando algoritmo genético. Para isso descreva a forma de representação do problema em cromossomos, a função de ajuste (fitness) e os operadores genéticos (mutação, crossover e o mecanismo de seleção).  
Assuma valores razoáveis para os hiperparâmetros comentando a sua escolha. Depois experimente alterá-los e reporte o que acontece

In [7]:
# Criando a classe item para ficar empacotar as informações dos itens e acessar de forma padronizada
class Item:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value

In [8]:
items = [
    Item("Barraca", 3.5, 150.00),
    Item("Saco de dormir", 2.0, 100.00),
    Item("Isolante térmico", 0.5, 50.00),
    Item("Colchão inflável", 1.0, 80.00),
    Item("Lanterna", 0.2, 30.00),
    Item("Kit de primeiros socorros", 0.5, 20.00),
    Item("Repelente de insetos", 0.1, 15.00),
    Item("Protetor solar", 0.2, 20.00),
    Item("Canivete", 0.1, 10.00),
    Item("Mapa e bússola", 0.3, 25.00),
    Item("Garrafa de água", 1.8, 15.00),
    Item("Filtro de água", 0.5, 50.00),
    Item("Comida (ração liofilizada)", 3.0, 50.00),
    Item("Fogão de camping", 1.5, 70.00),
    Item("Botijão de gás", 1.2, 30.00),
    Item("Prato, talheres e caneca", 0.5, 20.00),
    Item("Roupas (conjunto)", 1.5, 80.00),
    Item("Calçados (botas)", 2.0, 120.00),
    Item("Toalha", 0.5, 20.00),
    Item("Kit de higiene pessoal", 0.5, 30.00)
]

## Cromossomos

Vamos criar o cromossomo como uma classe, onde poderemos definir seu tamanho, dessa forma o tamanho do cromossomo poderá ser utilizado como um hiperparametro.  
Vamos também definir como método uma função fitness, podendo assim calcular qual seu fitness

In [9]:
class Chromosome:
    def __init__(self, length, items, max_weight, random_state=None):
        self.length = length
        self.items = items
        self.max_weight = max_weight
        self.random_state = random_state
        random.seed(random_state)
        self.genes = [random.randint(0,1) for _ in range(length)]

    def get_metrics(self):

        weight = sum([self.items[index].weight for index, gene in enumerate(self.genes) if gene == 1])

        if weight <= self.max_weight:
            fitness = sum([self.items[index].value for index, gene in enumerate(self.genes) if gene == 1])
        else:
            fitness = 0

        return fitness, weight


In [11]:
# Testando Criação dos Genes e verificando random_state para replicabilidade
print('With random state: ', Chromosome(10, items, 15, 777).genes)
print('With random state: ', Chromosome(15, items, 15, 777).genes)
print('Without random state: ', Chromosome(15, items, 15).genes)
print('Without random state: ', Chromosome(15, items, 15).genes)

With random state:  [0, 1, 1, 1, 1, 1, 0, 1, 1, 1]
With random state:  [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]
Without random state:  [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
Without random state:  [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]


Agora vamos criar funções capazes de realizar as seguintes ações:  
- Crossover
- Mutação

## Crossover
Na função de crossover implementada estamos utilizando um sorteio com probabildiade de 50% para selecionar quais genes do cromossomo 1 serão trocados com o do cromossomo 2.  
Após essa definição sorteamos a mesma quantidade de genes do cromossomo 2 para sofrerem o crossover

In [151]:
def crossover(chromosome_x, chromosome_y, crossover_prob,random_state = None):
    random.seed(random_state)
    genes_to_crossover_x = [i for i in range(chromosome_x.length) if random.uniform(0,1) <= crossover_prob]

    genes_to_crossover_y = list(range(chromosome_y.length))
    random.seed(random_state)
    random.shuffle(genes_to_crossover_y)
    genes_to_crossover_y = genes_to_crossover_y[:len(genes_to_crossover_x)]

    chromosome_crossover = copy.deepcopy(chromosome_x)
    for x, y in zip(genes_to_crossover_x, genes_to_crossover_y):
        chromosome_crossover.genes[x] = chromosome_y.genes[y]

    return chromosome_crossover

In [152]:
x, y = Chromosome(10, items, 15), Chromosome(10, items, 15)

In [157]:
x.genes, y.genes

([0, 1, 0, 1, 1, 0, 1, 1, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1])

In [155]:
crossover(x, y, 0.7).genes

[0, 1, 1, 1, 0, 0, 1, 1, 0, 1]

## Mutação
Para mutação estamos usando uma taxa fixa de 50% de chance de cada gene do cromossomo ser alterado

In [219]:
def mutate(chromosome, mutation_prob, random_state=None):
    random.seed(random_state)
    mutated_chromosome = copy.deepcopy(chromosome)
    mutated_chromosome.genes = [(gene+1)%2 if random.uniform(0,1) <= mutation_prob else gene for gene in chromosome.genes]

    return mutated_chromosome

In [220]:
x.genes

[0, 1, 1, 0, 1, 0, 1, 0, 1, 1]

In [224]:
mutate(x, 0.5, 777).genes

[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]

# O que ainda precisamos fazer....
Precisamos implementar uma forma de seleção de indivíduos entre gerações.
- Deixo como ideia implementar uma função que recebe uam lista de cromossomos, uma taxa de pais que vão para próxima geração, uma probabilidade de mutação e uma probabilidade de crossover. Com essas informações ele realiza uma seleção por roleta dos pais e depois com esses mesmos pais faz mutação e crossover se a probabilidade deles forem atingidas. - Implantado
- Tentem deixar o método de seleção variavel pelo tamanho da lista de cromossomos, dessa forma podemos hipertunar esse parametro.

In [225]:
def roulette_selection(chromosomes, fathers_next_gen, random_state=None):
    fitness_prob = [chromosome.get_metrics()[0] for chromosome in chromosomes]

    fitness_prob = [fitness/sum(fitness_prob) for fitness in fitness_prob]

    random.seed(random_state)
    fitness_prob = sorted([(fitness*random.uniform(0,1), index) for index, fitness in enumerate(fitness_prob)], reverse=True)
    fathers_chromosomes = [chromosomes[fitness_prob[index][1]] for index in range(fathers_next_gen)]

    return fathers_chromosomes

In [256]:
roulette_selection([x, y, x, y, x, y, y, y, y], 5, 777)

In [ ]:
def select(chromosomes, fathers_next_gen, mutation_prob, crossover_prob, random_state=None):

    fathers_chromosomes = roulette_selection(chromosomes, fathers_next_gen, random_state)
    qty_childs = len(chromosomes) - len(fathers_chromosomes)

    # crossover
    childs_chromosomes = []
    for _ in range(qty_childs):
        random.seed(random_state)
        chromosome_x = chromosomes[random.randint(0,len(chromosomes)-1)]
        random.seed(random_state)
        chromosome_y = chromosomes[random.randint(0,len(chromosomes)-1)]
        childs_chromosomes.append(crossover(chromosome_x, chromosome_y, crossover_prob, random_state))

    # child mutation
    for index, child_chromosome in enumerate(childs_chromosomes):
        childs_chromosomes[index] = mutate(child_chromosome, mutation_prob, random_state)

    return fathers_chromosomes + childs_chromosomes

In [263]:
chromosomes = [
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15),
    Chromosome(10, items, 15)
]

In [278]:
# 10 cromossomos, elitismo com 5 pais por roleta, 0.1% de probabilidade de mutação, 0.6% de probabilidade de crossover
select(chromosomes, 5, 0.1, 0.6, 777)

[<__main__.Chromosome object at 0x7fe3038cf350>, <__main__.Chromosome object at 0x7fe3038cc1d0>, <__main__.Chromosome object at 0x7fe3038cfad0>, <__main__.Chromosome object at 0x7fe3038ccf50>, <__main__.Chromosome object at 0x7fe3038cf140>, <__main__.Chromosome object at 0x7fe3200a5790>, <__main__.Chromosome object at 0x7fe320b68860>, <__main__.Chromosome object at 0x7fe30371f0e0>, <__main__.Chromosome object at 0x7fe30371f080>, <__main__.Chromosome object at 0x7fe30372dc40>]


In [ ]:
def log_data():
    pass

- Precisamos também de um loop que itere nas populações, e vá salvando/printando os resultados, seria legal conseguirmos plotar um grafico de linha de fitness no eixo y e gerações no eixo x, mostrando o aumento do fitness conforme passamos de geração

In [ ]:
def evolve():
    pass

Seria legal também ter alguma forma da gente trocar os hiperparametros para testar novas combinações e se elas tem muita diferença, exemplo:  
- Teste 1 - cromossomos = 10, tamanho = 10, taxa de pais = 50%, prob. mutação = 50%, prob. crossover = 50%, gerações = 100
- Teste 2 - cromossomos = 20, tamanho = 10, taxa de pais = 50%, prob. mutação = 50%, prob. crossover = 50%, gerações = 100
- Teste 3 - cromossomos = 10, tamanho = 10, taxa de pais = 50%, prob. mutação = 50%, prob. crossover = 50%, gerações = 1000

In [ ]:
def tune_hyperparams():
    pass